# Warming up with JAX

In [1]:
import numpy as np
import utils as ut
from tqdm.notebook import tqdm
import scipy as sc
from joblib import Parallel, delayed
from dawen.batched_inv_joblib import recompute_factors_batched

In [2]:
items=20108
userCnt , X= ut.load_xtx_binary(items)
XtX = (X.T @ X).toarray()
X_test = ut.load_xtx_binary_val()
users = 116677
nonzeros = X.nonzero()
nonzerosT = (X.T).nonzero()
get_ed_X = False
train_on_val = False
cg = True
if get_ed_X:
    print('Getting eigendecomposition of XtX')
    XtX = (X.T @ X).toarray()
    xvals, xvecs = np.linalg.eigh(XtX)
    np.save('xvals.npy', xvals)
    np.save('xvecs.npy', xvecs)
else:
    print('Loading eigendecomposition of XtX')
    xvals = np.load('xvals.npy')
    xvecs = np.load('xvecs.npy')
    #xvals_val = np.load('xvals_val.npy')
    #xvecs_val = np.load('xvecs_val.npy')


(116677, 20108)
Loading eigendecomposition of XtX


In [3]:
import numpy as np
from scipy import sparse

def compute_D_element(i, j, V, X, w, u, d):
    # Initialize the result with the identity matrix contribution
    result = 1 if i == j else 0
    
    # Compute the contribution from the matrix multiplication
    for k in range(d):
        for l in range(u):
            # Kronecker product terms
            kron_left = V[i % d, k] * X[l, i // d]
            kron_right = V[j % d, k] * X[l, j // d]
            
            # w is now a vector, so we directly access its element
            w_element = w[k*u + l]
            
            result += kron_left * w_element * kron_right

    return result

In [4]:
W = np.ones((users,items))
W[nonzeros] = 5
w = W.flatten('F')
V = np.random.randn(items,100)

In [5]:
test = compute_D_ij(0, 0, V, X, w)

IndexError: column index (20108) out of range

In [82]:
def iter_rows(S):
    """
    Helper function to iterate quickly over the data and indices of the
    rows of the S matrix. A naive implementation using indexing
    on S is much, much slower.
    """
    for i in range(S.shape[0]):
        lo, hi = S.indptr[i], S.indptr[i + 1]
        yield i, S.data[lo:hi], S.indices[lo:hi]


def recompute_factors(Y, S, lambda_reg, dtype='float32'):
    """
    recompute matrix X from Y.
    X = recompute_factors(Y, S, lambda_reg)
    This can also be used for the reverse operation as follows:
    Y = recompute_factors(X, ST, lambda_reg)

    The comments are in terms of X being the users and Y being the items.
    """
    m = S.shape[0]  # m = number of users
    f = Y.shape[1]  # f = number of factors
    # pre-compute this
    invYTYpI = np.linalg.inv(Y.T.dot(Y) + lambda_reg * np.eye(f))
    X_new = np.zeros((m, f), dtype=dtype)

    for k, s_u, i_u in iter_rows(S):
        Y_u = Y[i_u]  # exploit sparsity
        A = np.dot(s_u, Y_u)

        X_new[k] = A@(invYTYpI)
    return X_new

In [106]:
def get_V_cg(U,V,WXT,weight,X=X):
    nonzeros = WXT.nonzero()
    XU = X@U
    UXtXU = XU.T @ XU + eye
    pc = np.linalg.inv(UXtXU)
    VUX = V@XU.T
    VUX[nonzeros] *= weight
    r = WXT @ XU - (lam * V + VUX@XU)
    z = r @ pc
    p = z.copy()
    for _ in tqdm(range(210)):
        PUX = p@XU.T
        PUX[nonzeros] *= weight
        Ap = (PUX @ XU + lam*p)
        rtz = np.dot(r.flatten('F'),z.flatten('F'))
        alpha = rtz / np.dot(p.flatten('F'), Ap.flatten('F'))
        V += alpha * p
        r -= alpha * Ap
        print(np.linalg.norm(r.flatten()))
        if np.linalg.norm(r.flatten()) < 1e-8:
            return V
        z = r @ pc
        beta = np.dot(r.flatten('F'),p.flatten('F')) / rtz
        p = z + beta * p


def get_wmf_U(weight,lam,d,items,X=X,xvals=xvals,xvecs=xvecs):
    nonzeroU = X.nonzero()
    u = U.flatten('F')
    VtV = V.T @ V
    vvals, vvecs = np.linalg.eigh(VtV)
    D_inv = 1 / (np.kron(vvals,xvals) + lam)
    XUV = X @ U @ V.T
    XUV[nonzeroU] = weight * XUV[nonzeroU]
    r = (X.T @ (W2X @ V)).flatten('F') - (X.T @ (XUV@V) + lam*U).flatten('F')
    z = (xvecs @ (D_inv * (xvecs.T @ r.reshape((items,d),order='F') @ vvecs).flatten('F')).reshape((items,d),order='F') @ vvecs.T).flatten('F')
    p = z
    for _ in tqdm(range(210)):
        XPV = X @ p.reshape((items,d),order='F') @ V.T
        XPV[nonzeroU] = weight * XPV[nonzeroU]
        Ap = (X.T @ (XPV @ V)).flatten('F') + lam * p
        rtz = np.dot(r,z)
        alpha = rtz / np.dot(p,Ap)
        u = u + alpha * p
        r_new = r - alpha * Ap
        DXRV = D_inv * (xvecs.T @ r_new.reshape((items,d),order='F') @ vvecs).flatten('F')
        z = (xvecs @ DXRV.reshape((items,d),order='F') @ vvecs.T).flatten('F')
        print(np.linalg.norm(r_new))
        if np.linalg.norm(r_new) < 1e-8:
            return u.reshape((items,d),order='F')
        beta = np.dot(r_new,z) / rtz
        p = z + beta * p
        r = r_new
    return u.reshape((items,d),order='F')

In [107]:
weight = 1
W2X = (weight)*X
lam = 1
d=100
U = np.random.normal(size = (items,d)) * 1 / np.sqrt(d**(3) * weight)
V = np.random.normal(size = (items,d)) * 1 / np.sqrt(d**(3) * weight)

In [108]:
eye = lam * np.eye(d)
U = get_wmf_U(weight,lam,d,items,X=X,xvals=xvals,xvecs=xvecs)

  0%|          | 0/210 [00:00<?, ?it/s]

7.604862906999857e-11


In [109]:
XUV = X@U@V.T
XUV[X.nonzero()] *= 1
print(np.linalg.norm(lam*U - X.T @ (W2X@V) + X.T @ (XUV@V)))

7.509964055166179e-11


In [110]:
V = get_V_cg(U,V,W2X.T,weight,X=X)

  0%|          | 0/210 [00:00<?, ?it/s]

7.757012910048986e-10


In [111]:
XU = X@U
VUX = V@XU.T
VUX[X.T.nonzero()] *= 1
np.linalg.norm(lam * V - W2X.T@XU+ VUX @ XU)

7.81665807246999e-10

In [ ]:
#def get_V_cg2(U,V,WXT,weight,X=X):
#     nonzeros = WXT.nonzero()
#     XU = X@U
#     UXtXU = XU.T @ XU + eye
#     pc = np.linalg.inv(UXtXU)
#     VUX = V@XU.T
#     VUX[nonzeros] *= weight
#     r = WXT @ XU - (lam * V + VUX@XU)
#     z = r @ pc
#     p = z.copy()
#     for _ in tqdm(range(210)):
#         PUX = p@XU.T
#         PUX[nonzeros] *= weight
#         Ap = (PUX @ XU + lam*p)
#         rtz = np.dot(r.flatten('F'),z.flatten('F'))
#         alpha = rtz / np.dot(p.flatten('F'), Ap.flatten('F'))
#         V += alpha * p
#         r -= alpha * Ap
#         print(np.linalg.norm(r.flatten()))
#         if np.linalg.norm(r.flatten()) < 1e-12:
#             return V
#         z = r @ pc
#         beta = np.dot(r.flatten('F'),p.flatten('F')) / rtz
#         p = z + beta * p


# def get_wmf_U(weight,lam,d,items,X=X,xvals=xvals,xvecs=xvecs):
#     nonzeroU = X.nonzero()
#     u = U.flatten('F')
#     VtV = V.T @ V
#     vvals, vvecs = np.linalg.eigh(VtV)
#     D_inv = 1 / (np.kron(vvals,xvals) + lam)
#     XUV = X @ U @ V.T
#     XUV[nonzeroU] = weight * XUV[nonzeroU]
#     r = (X.T @ (W2X @ V)).flatten('F') - (X.T @ (XUV@V) + lam*U).flatten('F')
#     z = (xvecs @ (D_inv * (xvecs.T @ r.reshape((items,d),order='F') @ vvecs).flatten('F')).reshape((items,d),order='F') @ vvecs.T).flatten('F')
#     p = z
#     for _ in tqdm(range(210)):
#         XPV = X @ p.reshape((items,d),order='F') @ V.T
#         XPV[nonzeroU] = weight * XPV[nonzeroU]
#         Ap = (X.T @ (XPV @ V)).flatten('F') + lam * p
#         rtz = np.dot(r,z)
#         alpha = rtz / np.dot(p,Ap)
#         u = u + alpha * p
#         r_new = r - alpha * Ap
#         DXRV = D_inv * (xvecs.T @ r_new.reshape((items,d),order='F') @ vvecs).flatten('F')
#         z = (xvecs @ DXRV.reshape((items,d),order='F') @ vvecs.T).flatten('F')
#         print(np.linalg.norm(r_new))
#         if np.linalg.norm(r_new) < 1e-8:
#             return u.reshape((items,d),order='F')
#         beta = np.dot(r_new,z) / rtz
#         p = z + beta * p
#         r = r_new
#     return u.reshape((items,d),order='F')


def get_U_cg(U,V,WX,X=X,xvals=xvals,xvecs=xvecs,XtX=XtX):
    u = U.flatten('F')
    VtV = V.T @ V
    vvals, vvecs = np.linalg.eigh(VtV)
    D_inv = 1 / (np.kron(vvals,xvals) + lam)
    r = ((X.T @ (W2X @ V)) - (XtX @ U @ VtV + X.T @ ((WX.multiply(X@U@V.T))@V) + lam*U)).flatten('F')
    z = (xvecs @ (D_inv * (xvecs.T @ r.reshape((items,d),order='F') @ vvecs).flatten('F')).reshape((items,d),order='F') @ vvecs.T).flatten('F')
    p = z
    for _ in tqdm(range(210)):
        Ap = (XtX @ p.reshape((items,d),order='F') @ VtV + X.T @ ((WX.multiply(X@p.reshape((items,d),order='F')@V.T))@V) + lam*p.reshape((items,d),order='F')).flatten('F')
        rtz = np.dot(r,z)
        alpha = rtz / np.dot(p,Ap)
        u += alpha * p
        r -= alpha * Ap
        DXRV = D_inv * (xvecs.T @ r.reshape((items,d),order='F') @ vvecs).flatten('F')
        z = (xvecs @ DXRV.reshape((items,d),order='F') @ vvecs.T)
        print(np.linalg.norm(r))
        if np.linalg.norm(r) < 1e-8:
            return u.reshape((items,d),order='F')
        beta = np.dot(r,z) / rtz
        p = z + beta * p
    return u.reshape((items,d),order='F')

# def get_wmf_U(weight,lam,d,items,X=X,xvals=xvals,xvecs=xvecs):
#     nonzeroU = X.nonzero()
#     u = U.flatten('F')
#     VtV = V.T @ V
#     vvals, vvecs = np.linalg.eigh(VtV)
#     D_inv = 1 / (np.kron(vvals,xvals) + lam)
#     XUV = X @ U @ V.T
#     XUV[nonzeroU] = weight * XUV[nonzeroU]
#     r = (X.T @ (W2X @ V)).flatten('F') - (X.T @ (XUV@V) + lam*U).flatten('F')
#     z = (xvecs @ (D_inv * (xvecs.T @ r.reshape((items,d),order='F') @ vvecs).flatten('F')).reshape((items,d),order='F') @ vvecs.T).flatten('F')
#     p = z
#     for _ in tqdm(range(210)):
#         XPV = X @ p.reshape((items,d),order='F') @ V.T
#         XPV[nonzeroU] = weight * XPV[nonzeroU]
#         Ap = (X.T @ (XPV @ V)).flatten('F') + lam * p
#         rtz = np.dot(r,z)
#         alpha = rtz / np.dot(p,Ap)
#         u = u + alpha * p
#         r_new = r - alpha * Ap
#         DXRV = D_inv * (xvecs.T @ r_new.reshape((items,d),order='F') @ vvecs).flatten('F')
#         z = (xvecs @ DXRV.reshape((items,d),order='F') @ vvecs.T).flatten('F')
#         print(np.linalg.norm(r_new))
#         if np.linalg.norm(r_new) < 1e-8:
#             return u.reshape((items,d),order='F')
#         beta = np.dot(r_new,z) / rtz
#         p = z + beta * p
#         r = r_new
#     return u.reshape((items,d),order='F')


def get_V_cg(U,V,WXT,X=X):
    XU = X@U
    UXtXU = XU.T @ XU
    UXtXU_eye = UXtXU + eye
    pc = np.linalg.inv(UXtXU_eye)
    r = WXT @ XU - (V@UXtXU + lam*V + (WXT.multiply(V@XU.T))@XU)
    z = r @ pc
    p = z.copy()
    for _ in tqdm(range(210)):
        Ap = (p@UXtXU + lam*p + (WXT.multiply(p@XU.T))@XU)
        rtz = np.dot(r.flatten('F'),z.flatten('F'))
        alpha = rtz / np.dot(p.flatten('F'), Ap.flatten('F'))
        V += alpha * p
        r -= alpha * Ap
        print(np.linalg.norm(r.flatten()))
        if np.linalg.norm(r.flatten()) < 1e-8:
            return V
        z = r @ pc
        beta = np.dot(r.flatten('F'),p.flatten('F')) / rtz
        p = z + beta * p
    return V




    